# Capstone Project: Robo Trader

### Deboleena Mukhopadhyay

In [46]:
# Input: Company name

company = 'Amazon.com, Inc.'
print(f"✅ company is {company}.")

✅ company is Amazon.com, Inc..


In [47]:
# Imports

import json
import numpy as np
import os
import pandas as pd
import requests
import yfinance as yf

from google.adk.agents import Agent, LlmAgent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool
from google.genai import types
from kaggle_secrets import UserSecretsClient
from typing import Dict, Any

print("✅ Imports successful.")

✅ Imports successful.


In [48]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [49]:
# Authentication key for Google APIs.
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [50]:
# Retry configuration.

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)
print("✅ retry_config is defined.")

✅ retry_config is defined.


In [51]:
# ResearchAgent: Defined as a Wall Street analyst powered by Gemini 2.5 Flash Lite. 
# Its role is to use Google Search to summarize the latest annual earnings report and provide the consensus analyst price target and range for a stock. 
# The results are passed as "concensus_research".

research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are a Wall Street researcher, must do the following: 
        1. Summarize the latest earnings report of the current year for the company.
        2. Your must provide a concensus analysts price target for the stock with a range in your report.
    """,
    tools=[google_search],
    output_key="concensus_research", 
)

print("✅ ResearchAgent created.")

✅ ResearchAgent created.


In [52]:
# TickerAgent: Uses Gemini 2.5 Flash Lite and Google Search to find a company's stock ticker symbol. 
# It strictly returns the ticker symbol as a single word, passing the output under the key "ticker_symbol".

ticker_agent = Agent(
    name="TickerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction= "You must provide the ticker symbol for the company. Just return the ticker symbol as a single word.",
    
    tools=[google_search],
    output_key="ticker_symbol",  # The result of this agent will be stored in the session state with this key.
)

print(f"✅ TickerAgent created.  ")

✅ TickerAgent created.  


In [54]:
# Utility function for fetching the last one month's historical stock data (Open, High, Low, Close, Volume) for a given ticker 
# using the yfinance library that returns the data structred with a Pandas DataFrame of as a string.

def get_historical_data(ticker: str) -> dict:
    """Fetches historical data for the specified ticker.
    
    Args:
        ticker: The stock ticker symbol.
        
    Returns:
        Dictionary with status and historical data.
        Success: {'status': 'success', 'data': A multi-line string containing the simulated historical data table}
        Error: {"status": "error", "error_message": Exception message. }
    """
    try:
        tickerSym = yf.Ticker(ticker)                                     
        data_df = tickerSym.history(period="1mo")
        if data_df.shape[0] > 0:
            return {'status': 'success', 'data': data_df.to_string() }
        else:
            return {'status': 'error', 'data': '' }
    except Exception as e:
        return {'status': 'error', 'error_message': str(e) }

In [55]:
# FinanceDataAgent: It chains two tools -- [i] the ticker symbol from TickerAgent as a AgentTool, and historical data using get_historical_data() tool. 
# It then returns the result in a tabular format.

finance_data_agent = Agent(
    name="FinanceDataAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Your must provide historical trading data for the company by doing the following:

        1. Use the ticker_agent to get ticker_symbol for the company
        2. Use `get_historical_data()` with the ticker symbol to fetch historical trading data
        3. Check the "status" field in get_historical_data() response for errors
        4. If the status is "success", return the data in a tabular format
        5. If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_historical_data, AgentTool(ticker_agent)],
    output_key="financial_data"
)

print("✅ FinanceDataAgent created.")

✅ FinanceDataAgent created.


In [56]:
# The ParallelResearchTeam runs ResearchAgent and FinanceDataAgent simultaneously.

parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[research_agent, finance_data_agent],
)

print("✅ ParallelResearchTeam created.")

✅ ParallelResearchTeam created.


In [57]:
# DecisionMakingAgent: Defined as a Wall Street researcher using Gemini 2.5 Flash Lite to synthesize concensus_research and financial_data. 
# It then provides a clear BUY, HOLD, or SELL stock purchase recommendation, passing the output as "decision".

decision_making_agent = Agent(
    name="DecisionMakingAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    
    instruction="""You are a Wall Street researcher. You must
    1. Report the following:
    
        **Concensus research:**
        {concensus_research}
     
        **Financial data:**
        {financial_data}

    2. Provide a stock purchase decision as BUY or HOLD or SELL based on {concensus_research} and {financial_data}
    """,
    output_key="decision",
)

print("✅ DecisionMakingAgent is created.")

✅ DecisionMakingAgent is created.


In [58]:
# This is a MOCK-UP. Real trading requires valid API registration,
# OAuth 2.0 token management, and strict compliance with Schwab's API documentation.

SCHWAB_API_BASE_URL = "https://api.schwabapi.com/v1"
# You must replace '{accountId}' in the actual path with your Schwab account ID.
ORDER_ENDPOINT = "/accounts/{accountId}/orders"

def mock_buy_stock_order(
    ticker: str,
    quantity: int = 10, 
    price_type: str = "MARKET",
    access_token: str = "eyJraWQiOiJmYWtlX2tleSIsImFsZyI6IlJTMjU2In0.mock.token", 
    account_id: str = "12345678",
) -> Dict[str, Any]:
    """
    Mocks the process of sending a 'Buy' order request to the Schwab Trading API.

    NOTE: This function does NOT perform a real trade. It only demonstrates 
    the structure of the request payload, headers, and API endpoint usage.

    Args:

    ticker: The stock ticker symbol.
    quantity: The number of shares to buy.
    price_type: The type of order (e.g., 'MARKET', 'LIMIT').
    access_token: Your securely managed OAuth 2.0 access token.
    account_id: Your Charles Schwab brokerage account ID.

    Returns:
        A dictionary representing the mock API response.
        Success: {"status": "success", "response": mock_response}
        Error: {"status": "error", "error_message": Error message}
    """
    
    # 1. Construct the API Endpoint
    api_url = SCHWAB_API_BASE_URL + ORDER_ENDPOINT.format(accountId=account_id)

    # 2. Define Request Headers (Crucial for Authentication and Content Type)
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }

    # 3. Define the Order Payload (This is the most detailed part)
    # The structure must precisely match Schwab's required JSON schema for orders.
    order_payload = {
        "orderType": price_type.upper(),
        "session": "NORMAL",  # Regular market hours
        "duration": "DAY",    # Good for the day
        # "complexOrderStrategyType": "NONE",
        "orderLegCollection": [
            {
                "instruction": "BUY",
                "quantity": quantity,
                "instrument": {
                    "symbol": ticker.upper(),
                    "assetType": "EQUITY"
                }
            }
        ],
        # Add price details if it's a LIMIT order
        # "price": 150.00 if price_type.upper() == 'LIMIT' else None,
    }

    print("--- Sending Mock Order Request ---")
    print(f"URL: {api_url}")
    print(f"Payload (Partial): {json.dumps(order_payload, indent=2)}")
    
    # --- MOCK NETWORK CALL ---
    # In a real application, you would use:
    # try:
    #     response = requests.post(api_url, headers=headers, json=order_payload)
    #     response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    #     return response.json()
    # except requests.exceptions.RequestException as e:
    #     return {"error": str(e)}

    # Since we are mocking, we return a simulated success response:
    mock_response = {
        "status": "Success",
        "orderId": "123456789",
        "message": f"Successfully placed {price_type.upper()} BUY order for {quantity} shares of {ticker}.",
        "submittedTime": "2025-11-23T16:00:00Z"
    }
    return {
        'status': 'success',
        'response': mock_response,
    }

In [59]:
# MockBuyStockAgent: It uses Gemini 2.5 Flash Lite to execute a mock trade for 10 shares of a specified company. 
# It calls the mock_buy_stock_order() tool and passes the result of the order in JSON format.

mock_buy_stock_agent = Agent(
    name="MockBuyStockAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=f"""Your job is to buy the stock of the {company} of quantity 10.

    1. Use `mock_buy_stock_order()` with the ticker symbol to perform the trading
    2. Return json.dumps(order_result, indent=4)
    """,
    tools=[mock_buy_stock_order],
    output_key="order_result"
)

print("✅ MockBuyStockAgent is created.")

✅ MockBuyStockAgent is created.


In [60]:
# ExecutionAgent: It acts on the upstream {decision} and calls the MockBuyStockAgent to buy 10 stocks only if the decision is BUY.
# Otherwise, it reports the full decision.

execution_agent = Agent(
    name="ExecutionAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are an agent for executing stock purchase based on the {decision} of from decision_making_agent.

        1. If the {decision} of decision_making_agent contains Trade Decision: BUY, buy 10 stocks using the mock_buy_stock_agent.
        2. If the {decision} of decision_making_agent contains Trade Decision: HOLD or SELL, report the decision of the decision_agent.
    
        You must show the full report from decision_making_agent.
    """,
    tools=[AgentTool(mock_buy_stock_agent)],
    output_key="execution_result"
)

print("✅ ExecutionAgent is created.")

✅ ExecutionAgent is created.


In [61]:
# RootAgent: It uses a SequentialAgent to define the high-level workflow: Runs the parallel team first, then runs the DecisionMakingAgent and
# the ExecutionAgent in sequence.

root_agent = SequentialAgent(
    name="RootAgent",
    sub_agents=[parallel_research_team, decision_making_agent, execution_agent],
)

root_agent_runner = InMemoryRunner(agent=root_agent)
response = await root_agent_runner.run_debug(f"{company}")


 ### Created new session: debug_session_id

User > Amazon.com, Inc.


FinanceDataAgent > The historical data for Amazon.com, Inc. (AMZN) is as follows:

| Date                 |   Open |    High |    Low |    Close |    Volume |   Dividends |   Stock Splits |
|:---------------------|-------:|--------:|-------:|---------:|----------:|------------:|---------------:|
| 2025-10-27 00:00:00-04:00 | 227.66 | 228.399994 | 225.539993 | 226.970001 |  38267000 |          0 |              0 |
| 2025-10-28 00:00:00-04:00 | 228.22 | 231.490005 | 226.210007 | 229.250000 |  47100000 |          0 |              0 |
| 2025-10-29 00:00:00-04:00 | 231.67 | 232.820007 | 227.759995 | 230.300003 |  52036200 |          0 |              0 |
| 2025-10-30 00:00:00-04:00 | 227.06 | 228.440002 | 222.750000 | 222.860001 | 102252900 |          0 |              0 |
| 2025-10-31 00:00:00-04:00 | 250.1  | 250.5    | 243.979996 | 244.220001 | 166340800 |          0 |              0 |
| 2025-11-03 00:00:00-05:00 | 255.36 | 258.6    | 252.899994 | 254.0    |  95997800 |          0 |       

--- Sending Mock Order Request ---
URL: https://api.schwabapi.com/v1/accounts/12345678/orders
Payload (Partial): {
  "orderType": "MARKET",
  "session": "NORMAL",
  "duration": "DAY",
  "orderLegCollection": [
    {
      "instruction": "BUY",
      "quantity": 10,
      "instrument": {
        "symbol": "AMZN",
        "assetType": "EQUITY"
      }
    }
  ]
}
ExecutionAgent > I have placed a mock buy order for 10 shares of AMZN. The order has been placed as a MARKET BUY order and the order ID is 123456789.
